In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, precision_score



In [7]:
df.head()

,Voltage
0,234.07
1,232.16
2,232.16
3,232.16
4,232.43


In [8]:
# Step 1: Load dataset
file_path = "voltage_data electric water heater.csv"  # Update with correct path if needed
df = pd.read_csv(file_path)



In [9]:
# Step 2: Check dataset structure
print(df.head())
print(df.info())



   Voltage
0   234.07
1   232.16
2   232.16
3   232.16
4   232.43
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Voltage  1000 non-null   float64
dtypes: float64(1)
memory usage: 7.9 KB
None


In [14]:
print(df.head())  # Should show both Voltage & Status columns
print(df["Status"].value_counts())

   Voltage  Status
0   234.07       1
1   232.16       1
2   232.16       1
3   232.16       1
4   232.43       1
1    566
0    434
Name: Status, dtype: int64


In [13]:
# Step 3: Check label distribution
df["Status"] = df["Voltage"].apply(lambda x: 1 if x >= 230 else 0)
df.rename(columns={"Label": "Status"}, inplace=True)
print("Label Distribution Before Augmentation:\n", df['Status'].value_counts())


Label Distribution Before Augmentation:
 1    566
0    434
Name: Status, dtype: int64


In [17]:
# Step 4: Handle missing class (create synthetic high-voltage samples)
low_count = len(df[df["Status"] == 0])
high_count = len(df[df["Status"] == 1])

if high_count == 0:
    print("⚠ No high-voltage samples found. Generating synthetic data...")
    high_voltage_values = np.linspace(df["Voltage"].max() * 1.1, df["Voltage"].max() * 1.2, num=low_count)
    high_voltage_df = pd.DataFrame({'Voltage': high_voltage_values, 'Status': 1})
    df = pd.concat([df, high_voltage_df], ignore_index=True)

# Step 5: Check label distribution after augmentation
print("Label Distribution After Augmentation:\n", df['Status'].value_counts())

# Step 6: Prepare features (X) and labels (y)
X = df[['Voltage']].values  # Using Voltage as input feature
y = df['Status'].values  # Target variable

# Step 7: Normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

# Step 8: Reshape data for LSTM (samples, time steps, features)
X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Step 9: Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=42, stratify=y)

# Step 10: Build LSTM Model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(1, 1)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 11: Train model
model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_data=(X_test, y_test))

# Step 12: Make predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Step 13: Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, zero_division=1)
precision = precision_score(y_test, y_pred, zero_division=1)

# Step 14: Display evaluation metrics
print("\n🔹 LSTM Model Evaluation 🔹")
print(f"✅ Mean Squared Error (MSE): {mse:.4f}")
print(f"✅ Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"✅ Accuracy: {accuracy:.4f}")
print(f"✅ F1-Score: {f1:.4f}")
print(f"✅ Precision: {precision:.4f}")


Label Distribution After Augmentation:
 1    566
0    434
Name: Status, dtype: int64
Epoch 1/1000
50/50 [==============================] - 13s 64ms/step - loss: 0.6883 - accuracy: 0.5612 - val_loss: 0.6834 - val_accuracy: 0.5650
Epoch 2/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.6809 - accuracy: 0.5663 - val_loss: 0.6804 - val_accuracy: 0.5650
Epoch 3/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.6792 - accuracy: 0.5663 - val_loss: 0.6789 - val_accuracy: 0.5650
Epoch 4/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.6771 - accuracy: 0.5663 - val_loss: 0.6780 - val_accuracy: 0.5650
Epoch 5/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.6750 - accuracy: 0.5663 - val_loss: 0.6768 - val_accuracy: 0.5650
Epoch 6/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.6743 - accuracy: 0.5900 - val_loss: 0.6757 - val_accuracy: 0.6050
Epoch 7/1000
50/50 [==============================] - 1s 

50/50 [==============================] - 1s 11ms/step - loss: 0.4774 - accuracy: 0.7937 - val_loss: 0.4835 - val_accuracy: 0.8700
Epoch 58/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.4723 - accuracy: 0.7675 - val_loss: 0.4751 - val_accuracy: 0.7300
Epoch 59/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.4857 - accuracy: 0.7575 - val_loss: 0.4819 - val_accuracy: 0.7150
Epoch 60/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.4640 - accuracy: 0.7875 - val_loss: 0.4770 - val_accuracy: 0.7150
Epoch 61/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.4810 - accuracy: 0.7462 - val_loss: 0.4937 - val_accuracy: 0.7000
Epoch 62/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.4709 - accuracy: 0.7862 - val_loss: 0.4819 - val_accuracy: 0.9250
Epoch 63/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.4659 - accuracy: 0.7862 - val_loss: 0.4544 - val_accuracy: 0.7500
Epoch 

Epoch 114/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.2889 - accuracy: 0.8750 - val_loss: 0.2433 - val_accuracy: 1.0000
Epoch 115/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.2706 - accuracy: 0.8988 - val_loss: 0.2810 - val_accuracy: 0.8700
Epoch 116/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.2686 - accuracy: 0.8938 - val_loss: 0.2371 - val_accuracy: 0.8700
Epoch 117/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.2506 - accuracy: 0.8900 - val_loss: 0.2565 - val_accuracy: 0.8700
Epoch 118/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.2521 - accuracy: 0.9062 - val_loss: 0.2935 - val_accuracy: 0.8150
Epoch 119/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.2673 - accuracy: 0.8975 - val_loss: 0.2523 - val_accuracy: 0.9250
Epoch 120/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.2392 - accuracy: 0.9050 - val_loss: 0.2299 - val_ac

50/50 [==============================] - 0s 9ms/step - loss: 0.1887 - accuracy: 0.9262 - val_loss: 0.6234 - val_accuracy: 0.7600
Epoch 171/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.3757 - accuracy: 0.8500 - val_loss: 0.1489 - val_accuracy: 1.0000
Epoch 172/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.2400 - accuracy: 0.8838 - val_loss: 0.2194 - val_accuracy: 0.8700
Epoch 173/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.2711 - accuracy: 0.8763 - val_loss: 0.1500 - val_accuracy: 1.0000
Epoch 174/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.2015 - accuracy: 0.9075 - val_loss: 0.1575 - val_accuracy: 0.9250
Epoch 175/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.1895 - accuracy: 0.9137 - val_loss: 0.2709 - val_accuracy: 0.8700
Epoch 176/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1846 - accuracy: 0.9125 - val_loss: 0.1718 - val_accuracy: 0.9250
Epo

50/50 [==============================] - 0s 10ms/step - loss: 0.1995 - accuracy: 0.9137 - val_loss: 0.1271 - val_accuracy: 1.0000
Epoch 227/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1574 - accuracy: 0.9362 - val_loss: 0.1503 - val_accuracy: 0.8850
Epoch 228/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.2837 - accuracy: 0.8737 - val_loss: 0.1368 - val_accuracy: 0.9250
Epoch 229/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1787 - accuracy: 0.9175 - val_loss: 0.1375 - val_accuracy: 0.9250
Epoch 230/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1605 - accuracy: 0.9325 - val_loss: 0.1744 - val_accuracy: 0.8850
Epoch 231/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1598 - accuracy: 0.9250 - val_loss: 0.1244 - val_accuracy: 1.0000
Epoch 232/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1780 - accuracy: 0.9187 - val_loss: 0.1251 - val_accuracy: 1.0000
E

...

50/50 [==============================] - 0s 10ms/step - loss: 0.2210 - accuracy: 0.9075 - val_loss: 0.1483 - val_accuracy: 0.8850
Epoch 734/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1568 - accuracy: 0.9312 - val_loss: 0.3058 - val_accuracy: 0.8700
Epoch 735/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1758 - accuracy: 0.9287 - val_loss: 0.1478 - val_accuracy: 0.9250
Epoch 736/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1477 - accuracy: 0.9250 - val_loss: 0.1221 - val_accuracy: 0.8850
Epoch 737/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1298 - accuracy: 0.9400 - val_loss: 0.1199 - val_accuracy: 0.8850
Epoch 738/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1625 - accuracy: 0.9275 - val_loss: 0.1636 - val_accuracy: 0.8850
Epoch 739/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.3194 - accuracy: 0.8725 - val_loss: 0.1394 - val_accuracy: 0.9250
Epo

50/50 [==============================] - 1s 11ms/step - loss: 0.1440 - accuracy: 0.9350 - val_loss: 0.1306 - val_accuracy: 0.9250
Epoch 790/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.1792 - accuracy: 0.9362 - val_loss: 0.2248 - val_accuracy: 0.8750
Epoch 791/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1883 - accuracy: 0.9225 - val_loss: 0.1268 - val_accuracy: 0.9250
Epoch 792/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1266 - accuracy: 0.9438 - val_loss: 0.1099 - val_accuracy: 1.0000
Epoch 793/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1449 - accuracy: 0.9275 - val_loss: 0.2075 - val_accuracy: 0.8750
Epoch 794/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1768 - accuracy: 0.9300 - val_loss: 0.1111 - val_accuracy: 1.0000
Epoch 795/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1511 - accuracy: 0.9375 - val_loss: 0.1302 - val_accuracy: 0.8850


Epoch 846/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1887 - accuracy: 0.9175 - val_loss: 0.2365 - val_accuracy: 0.8700
Epoch 847/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.1640 - accuracy: 0.9187 - val_loss: 0.1569 - val_accuracy: 0.8850
Epoch 848/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1408 - accuracy: 0.9300 - val_loss: 0.1388 - val_accuracy: 0.8850
Epoch 849/1000
50/50 [==============================] - 1s 12ms/step - loss: 0.1191 - accuracy: 0.9475 - val_loss: 0.2317 - val_accuracy: 0.8700
Epoch 850/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1242 - accuracy: 0.9500 - val_loss: 0.4270 - val_accuracy: 0.8700
Epoch 851/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1478 - accuracy: 0.9362 - val_loss: 0.3313 - val_accuracy: 0.8200
Epoch 852/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.3598 - accuracy: 0.8687 - val_loss: 0.2332 - val_ac

50/50 [==============================] - 0s 10ms/step - loss: 0.1682 - accuracy: 0.9200 - val_loss: 0.1718 - val_accuracy: 0.8850
Epoch 903/1000
50/50 [==============================] - 1s 10ms/step - loss: 0.1350 - accuracy: 0.9475 - val_loss: 0.1040 - val_accuracy: 1.0000
Epoch 904/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.2182 - accuracy: 0.9013 - val_loss: 0.1657 - val_accuracy: 0.8850
Epoch 905/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1720 - accuracy: 0.9225 - val_loss: 0.1165 - val_accuracy: 1.0000
Epoch 906/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1680 - accuracy: 0.9200 - val_loss: 0.1234 - val_accuracy: 0.8850
Epoch 907/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.1901 - accuracy: 0.9137 - val_loss: 0.1649 - val_accuracy: 0.9250
Epoch 908/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1659 - accuracy: 0.9250 - val_loss: 0.1198 - val_accuracy: 0.8850


Epoch 959/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1090 - accuracy: 0.9650 - val_loss: 0.1041 - val_accuracy: 1.0000
Epoch 960/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1772 - accuracy: 0.9112 - val_loss: 0.4666 - val_accuracy: 0.8700
Epoch 961/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.2535 - accuracy: 0.8900 - val_loss: 0.1112 - val_accuracy: 1.0000
Epoch 962/1000
50/50 [==============================] - 0s 10ms/step - loss: 0.2163 - accuracy: 0.8913 - val_loss: 0.2219 - val_accuracy: 0.8750
Epoch 963/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1592 - accuracy: 0.9225 - val_loss: 0.1193 - val_accuracy: 0.8850
Epoch 964/1000
50/50 [==============================] - 1s 11ms/step - loss: 0.1452 - accuracy: 0.9287 - val_loss: 0.1664 - val_accuracy: 0.8850
Epoch 965/1000
50/50 [==============================] - 0s 9ms/step - loss: 0.1740 - accuracy: 0.9200 - val_loss: 0.1107 - val_accurac